In [20]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Introduction

This notebook is a companion to the paper "A machine learning approach to portfolio pricing and risk management for high-dimensional problems".

It describes how to generate all the input data necessary to run any risk estimation method on the same sample problems described in that paper.

## Pre-requisites


This notebook was executed using the following conda environment.


In [2]:
import pandas as pd
import numpy as np

In [3]:
def es_cal(vec: np.ndarray, *, nalpha: int=None, alpha: float=None)->np.ndarray:
    assert (nalpha is None) != (alpha is None)
    if nalpha is None:
        nalpha = int(len(vec) * alpha)
    ord_vec = np.sort(vec, axis=0)
    d_es = ord_vec[:nalpha].mean()
    u_es = ord_vec[-nalpha:].mean() 
    return np.array([u_es, d_es])

In [4]:
def calc_stats(data):
    assert data.columns.to_list() == ['id', 'Z']
    mean = data['Z'].mean()
    es = es_cal(data['Z'].values-mean, alpha=0.01)
    var = np.percentile(data['Z'].values-mean, q=[99., 1.])
    return pd.Series(np.array([ es[0], es[1], var[0], var[1], mean]), 
                     index=['upper_es', 'lower_es', 'upper_var', 'lower_var', 'pv'])

## European Call

### Benchmark values: large Monte Carlo simulation 

There are two files containing the distribution of the Monte Carlo estimation of the time-1 value of the European call $E_1[Z]=Z_1$. Each file contains 1 million simulations of $Z_1$.

The files are called `call_annual_5y_HW_1million_z1dist.csv` and `call_annual_40y_HW_1million_z1dist.csv`. They have one column `id` for the simulation id, and one column `Z` for $\hat{Z}_1$.

The function `calc_stats` can be used to derive the present value (pv), expected shortfal (es) and value-at-risk (var) of these empirical distributions.


#### 5-year European call

In [22]:
data5y.head()

,id,Z
0,0,49.172710
1,1,21.375768
2,2,45.087081
3,3,13.291789
4,4,22.913205


In [23]:
data5y = pd.read_csv('data/call_annual_5y_HW_1million_z1dist.csv')
data5y.columns = ['id', 'Z']
print(calc_stats(data5y))


upper_es     56.858767
lower_es    -20.300520
upper_var    45.942977
lower_var   -19.575289
pv           22.169061
dtype: float64


#### 40-year European call

In [6]:
data40y = pd.read_csv('data/call_annual_40y_HW_1million_z1dist.csv')
data40y.columns = ['id', 'Z']
print(calc_stats(data40y))


upper_es     62.620513
lower_es    -36.641604
upper_var    51.836707
lower_var   -33.498265
pv           67.548771
dtype: float64


#### Out-of-sample testing

The files above have been used as an out-of-sample test of the quality of risk estimates for the methods described in our paper, and can be used for the same purpose by anyone using the training data (below) for their own methods.

In order to compare against the `Z` values in the files above, it is necessary to use the `X` values contained in the files `call_annual_5y_HW_1million_outer_x.csv` and `call_annual_40y_HW_1million_outer_x.csv`. These files contain the simulated values of the independent random variable `X` (3 dimensional) for time-1.


In [7]:
inputs_5y = pd.read_csv('data/call_annual_5y_HW_1million_outer_x.csv')
inputs_5y.columns = ['id', 'X_1', 'X_2', 'X_3']
inputs_5y = inputs_5y.set_index('id')
inputs_5y

,X_1,X_2,X_3
id,,,
0,-0.778494,-0.191097,1.866854
1,-0.187797,-1.116563,0.228727
2,-1.299981,1.614254,1.800982
3,-0.484180,0.990760,-0.417332
4,-1.185936,-1.130459,0.603606
...,...,...,...
999995,0.431926,0.420062,-1.154773
999996,-0.480266,-0.377955,-0.573606
999997,0.633306,0.567887,0.172504


In [8]:
inputs_40y = pd.read_csv('data/call_annual_40y_HW_1million_outer_x.csv')
inputs_40y.columns = ['id', 'X_1', 'X_2', 'X_3']
inputs_40y = inputs_40y.set_index('id')
inputs_40y

,X_1,X_2,X_3
id,,,
0,-0.162761,0.400169,-0.592535
1,-1.386972,1.108603,-0.163673
2,0.017901,-0.082510,0.774258
3,1.230923,0.504737,-1.367800
4,-0.680401,-2.400506,-1.623261
...,...,...,...
999995,0.022034,1.962401,-1.622897
999996,-0.223926,0.744122,1.602962
999997,0.142804,0.276060,-0.253136


### Training and validation data

The files `call_annual_5y_HW_training_data.h5` and `call_annual_40y_HW_training_data.h5` contain the datasets with training and validation data. Each file contains 500,000 data points (full path simulations). 

They contain:
- the independent random numbers (`ind_Z` variables), 
- the correlated random numbers (`Z` variables), 
- the economic variables `cash` (cash account), `r` (Hull-White instantaneous rate), `S` (equity index),
- the european call discounted cash flows, `call_0`

#### 5-year European call

In [9]:
pd.read_hdf('data/call_annual_5y_HW_training_data.h5')

variable_name,ind_Z_0,ind_Z_1,ind_Z_2,Z_r_c_0,Z_r_c_1,Z_r_e_0,Z_r_e_1,cash_0,r_0,S_0,...,ind_Z_1,ind_Z_2,Z_r_c_0,Z_r_c_1,Z_r_e_0,Z_r_e_1,cash_0,r_0,S_0,call_0
time,1,1,1,1,1,1,1,1,1,1,...,5,5,5,5,5,5,5,5,5,5
simulation,,,,,,,,,,,,,,,,,,,,,
0,-0.282880,0.499004,0.544870,-0.282880,0.342471,-0.282880,0.477285,1.021699,0.017575,110.177698,...,1.352286,0.996021,-0.527155,1.024685,-0.527155,0.870466,1.081766,0.003863,65.797628,0.000000
1,-0.448382,-1.521676,-0.064078,-0.448382,-1.573240,-0.448382,-0.152460,1.012296,0.016259,96.245178,...,-0.905896,-0.393407,-0.818455,-1.159094,-0.818455,-0.549150,1.059274,0.014569,50.055867,0.000000
2,0.782956,-0.827774,-0.011047,0.782956,-0.441553,0.782956,0.145767,1.017840,0.026049,102.719976,...,0.260470,-0.543024,-0.340791,0.100847,-0.340791,-0.600211,1.099111,0.014020,72.504879,0.000000
3,-0.022226,0.061115,0.487093,-0.022226,0.046954,-0.022226,0.472806,1.020243,0.019647,109.922146,...,1.000588,1.139532,0.600097,1.157646,0.600097,1.236528,1.114913,0.024429,131.300119,31.300119
4,-1.831952,-1.044818,1.230380,-1.831952,-1.695086,-1.831952,0.839131,1.011700,0.005259,117.287580,...,0.326284,0.402505,-0.067721,0.271233,-0.067721,0.380828,1.074169,0.013394,100.892280,0.892280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,0.131900,-0.499997,0.178911,0.131900,-0.404290,0.131900,0.201676,1.018023,0.020873,103.893702,...,1.073090,1.580283,-0.900832,0.618465,-0.900832,1.368188,1.073290,0.003969,264.861146,164.861146
499996,0.583427,0.097945,0.188079,0.583427,0.324993,0.583427,0.300964,1.021613,0.024463,106.351113,...,-1.681462,-2.176043,-1.519805,-2.151691,-1.519805,-2.436039,1.095873,0.009072,88.819049,0.000000


#### 40-year European call

In [10]:
pd.read_hdf('data/call_annual_40y_HW_training_data.h5')

variable_name,ind_Z_0,ind_Z_1,ind_Z_2,Z_r_c_0,Z_r_c_1,Z_r_e_0,Z_r_e_1,cash_0,r_0,S_0,...,ind_Z_1,ind_Z_2,Z_r_c_0,Z_r_c_1,Z_r_e_0,Z_r_e_1,cash_0,r_0,S_0,call_0
time,1,1,1,1,1,1,1,1,1,1,...,40,40,40,40,40,40,40,40,40,40
simulation,,,,,,,,,,,,,,,,,,,,,
0,1.399089,0.551147,-0.213793,1.399089,1.068741,1.399089,0.070344,1.025287,0.030948,101.922408,...,1.479383,-1.070555,1.402064,1.919287,1.402064,-0.768513,2.374760,0.040063,114.893651,14.893651
1,0.549116,-0.077850,-0.517721,0.549116,0.150297,0.549116,-0.397437,1.020752,0.024190,92.408818,...,-0.372389,1.640378,0.464261,-0.153442,0.464261,1.700088,2.530741,0.032278,181.096042,81.096042
2,1.303224,-0.024055,-0.318352,1.303224,0.503750,1.303224,-0.051275,1.022495,0.030186,99.202274,...,-1.093735,-0.470308,0.206086,-0.917637,0.206086,-0.419589,2.116980,0.026697,78.980859,0.000000
3,1.674702,0.155966,-1.470336,1.674702,0.818337,1.674702,-1.105688,1.024049,0.033139,80.462936,...,-1.298699,-1.441421,-2.108882,-2.039111,-2.108882,-1.834075,2.473374,0.010474,30.393166,0.000000
4,1.414333,-1.712168,0.020991,1.414333,-0.996065,1.414333,0.303434,1.015120,0.031069,105.727337,...,-0.064840,-0.539983,0.905760,0.306082,0.905760,-0.347922,2.401563,0.026670,112.359207,12.359207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,-0.250084,1.011742,0.191182,-0.250084,0.824862,-0.250084,0.137303,1.024081,0.017836,103.174982,...,-0.028528,-0.505860,1.435825,0.553153,1.435825,-0.208474,2.133264,0.018977,49.359642,0.000000
499996,0.246626,0.120527,-1.012130,0.246626,0.209780,0.246626,-0.942356,1.021045,0.021785,82.890945,...,0.407685,-1.997253,0.691426,0.651979,0.691426,-1.818615,2.271234,0.027740,213.289989,113.289989


## Insurance example - variable annuity guarantee

### Benchmark values: large Monte Carlo simulation 2

There are two files containing the distribution of the Monte Carlo estimation of the time-1 value of the European call $E_1[Z]=Z_1$. Each contains 1 million simulations of $Z_1$.

The files are called `var_ann_annual_5y_HW_1million_z1dist.csv` and `var_ann_annual_40y_HW_1million_z1dist.csv`. They have one column `id` for the simulation id, and one column `Z` for $\hat{Z}_1$.

The function `calc_stats` can be used to derive the present value (pv), expected shortfal (es) and value-at-risk (var) of these empirical distributions.

These files haven been used as an out-of-sample test of the quality of risk estimates for the methods described in our paper, and can be used for the same purpose for anyone using the training data (below) for their own methods.

#### 5 years maturity 2

In [12]:
data5y = pd.read_csv('data/var_ann_annual_5y_HW_1million_z1dist.csv')
data5y.columns = ['id', 'Z']
print(calc_stats(data5y))


upper_es     1.673945e+06
lower_es    -2.719976e+06
upper_var    1.520818e+06
lower_var   -2.260103e+06
pv          -2.319814e+07
dtype: float64


#### 40 years maturity 2

In [13]:
data40y = pd.read_csv('data/var_ann_annual_40y_HW_1million_z1dist.csv')
data40y.columns = ['id', 'Z']
print(calc_stats(data5y))

upper_es     1.673945e+06
lower_es    -2.719976e+06
upper_var    1.520818e+06
lower_var   -2.260103e+06
pv          -2.319814e+07
dtype: float64


#### Out-of-sample testing 2

The files above have been used as an out-of-sample test of the quality of risk estimates for the methods described in our paper, and can be used for the same purpose by anyone using the training data (below) for their own methods.

In order to compare against the `Z` values in the files above, it is necessary to use the `X` values contained in the files `var_ann_annual_5y_HW_1million_outer_x.csv` and `var_ann_annual_40y_HW_1million_outer_x.csv`. These files contain the simulated values of the independent random variable `X` (3 dimensional) for time-1.

In [14]:
inputs_5y = pd.read_csv('data/var_ann_annual_5y_HW_1million_outer_x.csv')
inputs_5y.columns = ['id', 'X_1', 'X_2', 'X_3']
inputs_5y = inputs_5y.set_index('id')
inputs_5y

,X_1,X_2,X_3
id,,,
0,-0.198394,-1.551837,1.279520
1,0.218109,0.598863,0.329664
2,-0.597616,-1.405504,-1.563193
3,-0.148785,0.142795,-0.324114
4,0.199315,0.014020,0.225635
...,...,...,...
999995,0.230748,0.968792,0.918105
999996,-0.740157,-1.260582,-0.317590
999997,1.249531,-0.821595,2.440714


In [15]:
inputs_40y = pd.read_csv('data/var_ann_annual_40y_HW_1million_outer_x.csv')
inputs_40y.columns = ['id', 'X_1', 'X_2', 'X_3']
inputs_40y = inputs_40y.set_index('id')
inputs_40y

,X_1,X_2,X_3
id,,,
0,-0.658131,-0.832913,-1.936748
1,0.402110,-2.178649,0.163172
2,1.059748,-0.635210,0.094618
3,0.060273,-0.037635,0.662728
4,0.039287,-1.113265,-0.099547
...,...,...,...
999995,-0.764794,-0.588790,0.215711
999996,-0.776344,-2.286754,1.096989
999997,0.273163,-1.076658,-0.447088


### Training and validation data 2

The files `var_ann_annual_5y_HW_training_data.h5` and `var_ann_annual_40y_HW_training_data.h5` contain the datasets with training and validation data. Each file contains 500,000 data points (full path simulations).

They contain:
- the independent random numbers (`ind_Z` variables), 
- the correlated random numbers (`Z` variables), 
- the economic variables `cash` (cash account), `r` (Hull-White instantaneous rate), `S` (equity index), `RE`(real estate index), 
- the asset prices, `constP_x` (price of an x-years-to-maturity bond), `b_10` (price of a portfolio that invests on 10-year bonds on a yearly rolling basis), `b_20` (same as `b_10` but with 20-year bonds), `inv_port` (total value of the investment portfolio per policy),
- the demographic variables `qx_y` (death rate for a person aged y), `pop_L_x` (the number of surviving lives of age x), `pop_D_x` (the number of people that died aged x)
- `prod_f` (the product cash flow resulting from the payout of the investment fund) and `prod_g` (the product cash flow resulting from payment of the guarantee), `cumCF` (the cumulative value of all payments up to that point in time).

#### 5 years maturity 2

In [16]:
pd.read_hdf('data/var_ann_5y_training_data.h5')

variable_name,ind_Z_0,ind_Z_1,ind_Z_2,ind_Z_3,ind_Z_4,Z_r_c_0,Z_r_c_1,Z_r_e_0,Z_r_e_1,cash,...,pop_D_37,pop_D_38,pop_D_39,pop_D_40,b_10,b_20,inv_port,prod_g,prod_f,cumCF
time,1,1,1,1,1,1,1,1,1,1,...,5,5,5,5,5,5,5,5,5,5
simulation,,,,,,,,,,,,,,,,,,,,,
0,-1.453298,-0.410163,-0.583956,-0.507732,0.766054,-1.453298,-0.411421,-1.453298,-0.862817,1.019516,...,29.965025,29.656485,29.333848,29.011591,0.968686,0.882351,774.770566,0.0,-3.065932e+07,-2.926341e+07
1,-0.796042,0.081056,0.454518,0.119973,0.265633,-0.796042,0.080366,-0.796042,0.286126,1.020095,...,28.476991,28.150540,27.835717,27.526213,0.957269,0.838068,779.928101,0.0,-3.081812e+07,-2.889940e+07
2,-0.634515,0.748122,0.633288,-0.570726,-0.526051,-0.634515,0.747573,-0.634515,0.493590,1.020881,...,27.841542,27.551713,27.254464,26.955412,0.946847,0.825253,767.892169,0.0,-3.038967e+07,-2.853033e+07
3,0.919585,-0.960716,0.365450,-0.050631,1.024792,0.919585,-0.959919,0.919585,0.541984,1.018871,...,31.020301,30.699134,30.358915,30.022459,0.946567,0.814706,810.110332,0.0,-3.202746e+07,-2.986016e+07
4,0.677756,0.524198,0.264120,1.521236,-1.595933,0.677756,0.524785,0.677756,0.394335,1.020619,...,29.998793,29.669296,29.341107,29.005977,0.909820,0.746098,848.749628,0.0,-3.355271e+07,-3.097858e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,0.096409,0.377690,0.013272,-0.365144,-2.297315,0.096409,0.377773,0.096409,0.032286,1.020446,...,29.597421,29.270964,28.962976,28.643921,0.925705,0.778459,799.649445,0.0,-3.163706e+07,-2.937981e+07
499996,1.279687,-0.768120,0.385463,-1.455597,-0.306005,1.279687,-0.767012,1.279687,0.633612,1.019098,...,27.539476,27.241364,26.946179,26.655487,0.892946,0.706663,742.084864,0.0,-2.937662e+07,-2.681419e+07


#### 40 years maturity 2

Due to the large size of this dataset, the variables `qx` have been removed. The variables `L` and `D` are only shown in aggregate for all ages (separately for each point in time). Neither of those variables are used by any of the estimators described in the paper. They are just intermediate variables.

In [17]:
pd.read_hdf('data/var_ann_40y_training_data.h5')

variable_name,ind_Z_0,ind_Z_1,ind_Z_2,ind_Z_3,ind_Z_4,Z_r_c_0,Z_r_c_1,Z_r_e_0,Z_r_e_1,cash,...,pol_L,pol_D,pol_L,pol_D,pol_L,pol_D,pol_L,pol_D,pol_L,pol_D
time,1,1,1,1,1,1,1,1,1,1,...,36,36,37,37,38,38,39,39,40,40
simulation,,,,,,,,,,,,,,,,,,,,,
0,-0.008827,0.994270,-0.604614,-0.117463,-0.504662,-0.008827,0.994262,-0.008827,-0.594164,1.021172,...,16898.205794,869.279406,16267.274941,563.593809,15448.293479,747.819118,14532.672784,842.492016,13853.168364,608.524704
1,1.370963,1.520117,-0.740782,0.535237,-0.855994,1.370963,1.521303,1.370963,-0.451623,1.021793,...,16939.157561,809.199244,16088.728909,782.220056,15368.849793,650.482170,14672.131525,625.191908,13825.282213,774.360281
2,-0.023351,-0.454862,0.398487,-0.178909,0.571798,-0.023351,-0.454882,-0.023351,0.385766,1.019465,...,16587.367695,911.637890,15873.739126,651.129806,15192.745483,615.718178,14374.097079,751.256351,13450.623206,856.618896
3,0.391102,0.794949,0.939231,0.226972,-0.685326,0.391102,0.795288,0.391102,0.998475,1.020937,...,16949.732463,755.155197,16033.776001,847.067252,15285.107578,681.059359,14588.324296,627.636283,13775.309518,741.485555
4,0.124095,-0.944293,-1.478837,-0.628056,-1.604985,0.124095,-0.944185,0.124095,-1.424140,1.018890,...,16993.830161,701.540064,16238.677917,685.521196,15593.974871,573.156820,14531.136451,987.881718,13695.406024,764.817822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,0.614467,1.894647,-0.078790,0.692228,0.889426,0.614467,1.895178,0.614467,0.045696,1.022234,...,16261.394763,821.312433,15585.734190,618.694469,14755.508846,770.839143,14059.909260,636.635148,13223.067062,775.939713
499996,-1.169627,-0.531183,-0.197515,0.253500,0.278569,-1.169627,-0.532196,-1.169627,-0.427450,1.019374,...,17158.711467,636.155280,16221.478175,864.627064,15508.404178,641.728082,14734.273646,700.706680,13890.730622,768.809932
